# Introducción
El presente notebook corresponde al trabajo hecho en Hands On Machine Learning de Aurelion Gerón.
El pipeline que se va a seguir es el que se propone en el libro.


* 1- Entender el problema.
* 2- Obtener los datos.
* 3- Visualizar los datos.
* 4- Preparar los datos para usar con los algoritmos de ML.
* 5- Seleccionar un modelo y entrenarlo.
* 6- Retocar el modelo.
* 7- Presentar la solución
* 8- Lanzar a producción y mantener el sistema.

## 2- Obtener los datos

In [ ]:
import pandas as pd

In [ ]:
housing = pd.read_csv('D:\Programacion\Proyectos\Data Science\HandsOn_Machine_Learning\california_housing_prices\dataset\housing.csv')
housing.head()

In [ ]:
housing.info()

La base de datos esta completa. El unico tema es en **total_bedrooms** que aparentemente le faltan algunos registros. En los demas no hay ningun Null por lo que no vamos a tener que trabajar con eso. Son en su mayoría atributos numericos, el unico a considerar es cercanía con el oceano.

Podemos chequear en el caso de proximidad al oceano que contiene ese atributo.

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.describe()

Si tenemos en cuenta los quartiles y el std vemos que en varios casos parecen existir outliers. Podemos mencionar a **total_rooms, total_bedrooms, population, households y median_house_value**, este ultimo caso bastante extraño que termine en un numero tan entero.

Otra forma de visualizar la información con la que estamos trabajando es con gráficos.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.show()

Queda claro como **median_house_value y housing_median_age** estan topeados, es algo que vamos a tener que trabajar. Se pueden realizar dos cosas:
* Los sacamos del dataset.
* Recolectamos los datos.

Con respecto a las demás variables, en su mayoría tienen un sesgo hacia la derecha. En algunos casos puede ser complicado trajar con distribuciones de este estilo, ya que es indicio de valores atipicos. Además, los rangos de las variables distan mucho entre si, por lo que vamos a tener que hacer que re escalar.

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.20, random_state=42)

### Consideraciones
Cuando hacemos este tipo de splits tenemos que tener cuidado porque podríamos estar introduciendo un montón de **sampling bias**. Si el dataset es lo suficientemente grande, realizar un split de manera random no sería un inconveniente, pero si no lo es tenemos que considerar de tomar una muestra **estratificada** de nuestra población objetivo. Esto quiere decir que si estamos trabajando estudiando autos argentinos y sabemos que en total hay 54% rojos, 20% y el resto blancos, cuand tomemos nuestra muestra vamos a querer que el % de representación de cada categoría de color se mantenga, es decir, 54% rojo, 20% negro y el resto blanco.

### Como estratificar?
En este problema puede ocurrir que los expertos nos digan que el ingreso medio por casa es la variable mas importante para definir el precio de la casa. Si ese es el caso, puede que querramos que nuestro dataset sea lo suficientemente representativo de las distintas categorías de ingresos que disponemos. Si analizamos el histograma anterior, vemos que la mayoría de los datos se concentran entre 1.5 y 6, pero que también hay datos muy alejados de la media, en este caso, podemos usar **pd.cut** para armar las categorias

In [ ]:
import numpy as np

housing['income_cat'] = pd.cut(housing['median_income'], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5])
housing['income_cat'].hist()

Teniendo las categorías, ahora podemos estratificar nuestro dataset. Lo hacemos de la siguiente manera.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]


# Chequeamos si funciona
strat_train_set['income_cat'].value_counts() / len(strat_train_set)

Podemos hacer el mismo analisis en el dataset completo para ver si coinciden.

In [ ]:
housing['income_cat'].value_counts() / len(housing)

Queda claro que sí. Son casi identicos. Esto era lo que tratamos de lograr cuando estratificamos la muestra.

Ahora si queremos, podemos remover la columna de income_cat para que el dataset vuelva a ser el mismo que el original.

In [ ]:
for set_ in (strat_test_set, strat_train_set):
    set_.drop("income_cat", axis=1, inplace=True)

## 3- Visualizar los datos
Tenemos un conocimiento basico de los datos, ahora la idea es explorar un poco mas en profundidad y empezar a plotear.

Lo primero que tenemos que hacer es asegurarnos de que estamos trabajando con el dataset para train, y que dejamos de lado test.

In [ ]:
housing = strat_train_set.copy()

In [ ]:
housing.plot(kind='scatter', x='longitude', y='latitude')

No es que se puede visualizar mucho. Hay una opción **alpha** que suaviza los colores de los puntos del plot, la podemos usar para observar donde estan mayoritariamente concentrados los puntos.

In [ ]:
housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.1)

Se observa que las zonas con mayor concentración son la costa y el centro particularmente.

Ahora podemos empezar a agregarle un poco más de cosas al mapa. Ya sabemos donde están concentradas las casas, pero que podemos decir con respecto al precio y la población?

In [ ]:
housing.plot(kind='scatter', x='longitude', y='latitude', s=housing['population']/100, 
            c='median_house_value', cmap=plt.get_cmap('jet'), colorbar=True, figsize=(15,10), alpha=0.4)
plt.legend(["poblacion"])

> #### > TIP
> En el caso de la función plot de pandas, cuando usamos el arguemento **c** para el color, es opcional pasarle un array o directamente el nombre de la columna.

Como podemos observar en el grafico el area de la Bahía es la más cara. En cuanto a la población parece estar bastante más distribuida, de hecho, se notan algunos aglomerados donde el precio de las casas no es tan alto, igualmente, la densidad poblacional del area de la bahia es obvia.

Entonces, una de las principales conclusiones es que el precio de las casas está bastante relacionado con la locaclización y la densidad poblacional.

### Correlación

In [ ]:
corr_housing = housing.corr()

In [ ]:
corr_housing['median_house_value'].sort_values(ascending=False)

> #### > TIP
> La interpretación de una correlación negativa entre latitude y el precio de la casa podríamos interpretarla como que los precios tienden a subir un poco cuando nos movemos hacia el sur.

Lo mas prometedor en este caso claramente es el ingreso medio. Lo que podemos hacer para ver estos numeros de una manera visual es realizar un scatter de columnas contra columnas para ver que forma adquieren. En este caso no tendría sentido plotear todas, ya que serian mas de 100 graficos. Podemos usar algunas que pensamos pueden sernos util mas adelante.

In [ ]:
from pandas.plotting import scatter_matrix

atributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
scatter_matrix(housing[atributes], figsize=(12,10))
plt.show()

Hay algunas cosas a anotar en este dataset como datos de color. Lo más importante, es que el atributo que a priori mas relacion tiene para predecir el valor de una casa es el ingreso. Pero adicionalmente se pueden hacer algunos comentarios con respecto a los datos que arrojan los scatters. Primero, si se mira la relacion entre total de habitaciones y los años de la casa, parece haber una leve tendencia en que las casas mas nuevas tienen mas habitaciones. Segundo, que en este dataset en particular en las areas donde se detectaron ingresos medios son donde estan las casas con mayor numero de habitaciones.

Hagamos un zoom en el valor que es mas factible de utilizar para predecir el predecir el precio de las casas.

In [ ]:
housing.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.1)